In [2]:
import pandas as pd

# 1. 파일 로드 (파일명은 'pet_food_customer_orders.csv'를 사용합니다)
try:
    df = pd.read_csv('pet_food_customer_orders.csv')
except FileNotFoundError:
    print("오류: 'pet_food_customer_orders.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")
    exit()

# 2. 필요한 데이터프레임 필드 선택
# pet_has_active_subscription: 구독 활성화 여부
# total_minutes_on_website: 총 웹사이트 이용 시간
# total_web_sessions: 총 웹 세션 수 (듀링)
# pet_order_number: 총 주문 수
analysis_df = df[[
    'pet_has_active_subscription', 
    'total_minutes_on_website', 
    'total_web_sessions',  # 추가된 필드
    'pet_order_number'
]].copy()

# 3. 정기구독 여부(pet_has_active_subscription)를 기준으로 그룹화하고 평균 계산
# 정기구독(TRUE) 그룹과 비구독(FALSE) 그룹의 평균을 계산합니다.
grouped_analysis = analysis_df.groupby('pet_has_active_subscription').mean()

# 4. 결과 출력
print("=======================================================================")
print("          정기구독 여부에 따른 웹 활동 및 주문 수 비교 (평균)")
print("=======================================================================")
# 결과를 보기 쉽게 소수점 둘째 자리까지 반올림하여 출력합니다.
print(grouped_analysis.round(2))
print("\n")

# 5. 가설 검증 결과 정리 및 해석
print("---------------------------- 결과 해석 --------------------------------")

# 각 그룹의 평균값 저장
avg_time_sub = grouped_analysis.loc[True, 'total_minutes_on_website']
avg_time_non_sub = grouped_analysis.loc[False, 'total_minutes_on_website']
avg_sessions_sub = grouped_analysis.loc[True, 'total_web_sessions']
avg_sessions_non_sub = grouped_analysis.loc[False, 'total_web_sessions']
avg_orders_sub = grouped_analysis.loc[True, 'pet_order_number']
avg_orders_non_sub = grouped_analysis.loc[False, 'pet_order_number']


# 웹 세션 시간 가설 검증: 구독 고객의 웹 세션 시간이 짧은가?
time_result = "짧습니다" if avg_time_sub < avg_time_non_sub else "길거나 비슷합니다"
print(f"1. 평균 웹 세션 시간 (total_minutes_on_website): 구독 고객의 시간이 비구독 고객보다 {time_result} (가설: 짧을 것이다)")

# 웹 세션 수 가설 검증: 구독 고객의 웹 세션 수가 적은가?
# 짧은 세션 시간(효율적)과 적은 세션 수(방문 빈도 낮음)는 일반적으로 함께 나타납니다.
session_result = "적습니다" if avg_sessions_sub < avg_sessions_non_sub else "많거나 비슷합니다"
print(f"2. 평균 웹 세션 수 (total_web_sessions): 구독 고객의 횟수가 비구독 고객보다 {session_result} (가설: 적을 것이다)")

# 주문 수 가설 검증: 구독 고객의 주문 수가 많은가?
order_result = "많습니다" if avg_orders_sub > avg_orders_non_sub else "적거나 비슷합니다"
print(f"3. 평균 총 주문 수 (pet_order_number): 구독 고객의 주문 수가 비구독 고객보다 {order_result} (가설: 많을 것이다)")

print("-----------------------------------------------------------------------")

          정기구독 여부에 따른 웹 활동 및 주문 수 비교 (평균)
                             total_minutes_on_website  total_web_sessions  \
pet_has_active_subscription                                                 
False                                          353.02                6.92   
True                                           398.59                8.47   

                             pet_order_number  
pet_has_active_subscription                    
False                                    3.09  
True                                     3.71  


---------------------------- 결과 해석 --------------------------------
1. 평균 웹 세션 시간 (total_minutes_on_website): 구독 고객의 시간이 비구독 고객보다 길거나 비슷합니다 (가설: 짧을 것이다)
2. 평균 웹 세션 수 (total_web_sessions): 구독 고객의 횟수가 비구독 고객보다 많거나 비슷합니다 (가설: 적을 것이다)
3. 평균 총 주문 수 (pet_order_number): 구독 고객의 주문 수가 비구독 고객보다 많습니다 (가설: 많을 것이다)
-----------------------------------------------------------------------


In [3]:
import pandas as pd
import numpy as np

# 1. 파일 로드 (파일명은 'pet_food_customer_orders.csv'를 사용합니다)
try:
    df = pd.read_csv('pet_food_customer_orders.csv')
except FileNotFoundError:
    print("오류: 'pet_food_customer_orders.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")
    exit()

# 필요한 컬럼만 추출하여 복사
analysis_df = df[[
    'pet_food_tier', 
    'pet_health_issue_list', 
    'total_web_sessions', 
    'total_minutes_on_website', 
    'total_order_kcal',
    'neutered'
]].copy()

# -------------------------- 컬럼 생성 및 전처리 --------------------------

# A. Diet 사료 급여 여부 컬럼 생성 (diet_y)
# 'pet_food_tier'가 'diet'인 경우 True, 아니면 False
analysis_df['diet_y'] = (analysis_df['pet_food_tier'] == 'diet')

# B. 칼로리 수준 컬럼 생성 (kcal_level)
# total_order_kcal을 사용하여 평균을 기준으로 고/저칼로리 구분
mean_kcal = analysis_df['total_order_kcal'].mean()
analysis_df['kcal_level'] = analysis_df['total_order_kcal'].apply(
    lambda x: 'Low' if x < mean_kcal else 'High'
)

# C. 중성화 여부 컬럼 정리 (neutered_y)
# 'neutered' 컬럼의 값을 bool 타입으로 정리 (TRUE/FALSE 문자열을 불리언으로)
analysis_df['neutered_y'] = analysis_df['neutered'].astype(str).str.upper() == 'TRUE'

# -------------------------- 1차 그룹별 통계 분석 --------------------------

print("=======================================================================")
print("  Diet 사료 급여 여부 (diet_y)에 따른 주요 변수 평균 비교")
print("=======================================================================")

# diet_y (True/False) 기준으로 그룹화하고 웹 활동 및 칼로리 평균 계산
grouped_mean = analysis_df.groupby('diet_y')[[
    'total_web_sessions', 
    'total_minutes_on_website', 
    'total_order_kcal'
]].mean().round(2)

print(grouped_mean)
print("\n")

# -------------------------- 2차 교차 분석 (질병 및 중성화) --------------------------

print("=======================================================================")
print("  Diet 사료 급여 고객 특성 교차 분석")
print("=======================================================================")

# 1. 질병 유무 확인 (pet_health_issue_list가 NaN 또는 None이 아닌 경우)
analysis_df['has_health_issue'] = analysis_df['pet_health_issue_list'].notna() & (analysis_df['pet_health_issue_list'] != 'None')

# Diet 고객 중 질병이 있는 비율
diet_with_issue_ratio = analysis_df[analysis_df['diet_y']]['has_health_issue'].mean() * 100
non_diet_with_issue_ratio = analysis_df[~analysis_df['diet_y']]['has_health_issue'].mean() * 100

print(f"1. 질병 유무 ('has_health_issue') 비율:")
print(f"   - Diet 고객 중 질병이 있는 비율: {diet_with_issue_ratio:.2f}%")
print(f"   - Non-Diet 고객 중 질병이 있는 비율: {non_diet_with_issue_ratio:.2f}%")
print("   -> 가설 검증: Diet 고객의 질병 유무 비율이 Non-Diet 고객보다 높아야 가설이 지지됩니다.")
print("\n")


# 2. 칼로리 수준 확인 (Diet 고객의 칼로리가 낮은 비율)
# diet_y와 kcal_level 간의 교차표 생성
kcal_crosstab = pd.crosstab(analysis_df['diet_y'], analysis_df['kcal_level'], normalize='index').round(4) * 100

print("2. 칼로리 수준 ('kcal_level') 비율:")
print(kcal_crosstab)
print("   -> 가설 검증: Diet 고객(True) 행에서 'Low' 칼로리 비율이 높아야 가설이 지지됩니다.")
print("\n")

# 3. 중성화 여부 확인 (Diet 고객의 중성화 비율)
# diet_y와 neutered_y 간의 교차표 생성
neutered_crosstab = pd.crosstab(analysis_df['diet_y'], analysis_df['neutered_y'], normalize='index').round(4) * 100

print("3. 중성화 여부 ('neutered_y' - True/False) 비율:")
print(neutered_crosstab)
print("   -> 가설 검증: Diet 고객(True) 행에서 'True' (중성화) 비율이 높을 수 있습니다 (중성화는 체중 관리 및 질병과 관련).")
print("-----------------------------------------------------------------------")

  Diet 사료 급여 여부 (diet_y)에 따른 주요 변수 평균 비교
        total_web_sessions  total_minutes_on_website  total_order_kcal
diet_y                                                                
False                 7.96                    383.58          20790.57


  Diet 사료 급여 고객 특성 교차 분석
1. 질병 유무 ('has_health_issue') 비율:
   - Diet 고객 중 질병이 있는 비율: nan%
   - Non-Diet 고객 중 질병이 있는 비율: 48.36%
   -> 가설 검증: Diet 고객의 질병 유무 비율이 Non-Diet 고객보다 높아야 가설이 지지됩니다.


2. 칼로리 수준 ('kcal_level') 비율:
kcal_level   High    Low
diet_y                  
False       39.84  60.16
   -> 가설 검증: Diet 고객(True) 행에서 'Low' 칼로리 비율이 높아야 가설이 지지됩니다.


3. 중성화 여부 ('neutered_y' - True/False) 비율:
neutered_y  False  True 
diet_y                  
False        43.9   56.1
   -> 가설 검증: Diet 고객(True) 행에서 'True' (중성화) 비율이 높을 수 있습니다 (중성화는 체중 관리 및 질병과 관련).
-----------------------------------------------------------------------


In [14]:
import pandas as pd
import numpy as np
# statsmodels.formula.api를 임포트하여 포뮬러 방식 사용
import statsmodels.formula.api as smf 
from sklearn.preprocessing import StandardScaler

# 1. 파일 로드
try:
    df = pd.read_csv('pet_food_customer_orders.csv')
except FileNotFoundError:
    print("오류: 'pet_food_customer_orders.csv' 파일을 찾을 수 없습니다. 파일 경로를 확인해 주세요.")
    exit()

# -------------------------- 데이터 전처리 및 컬럼 생성 --------------------------

# A. 종속 변수 생성: Diet 사료 급여 여부 (diet_y)
df['diet_y'] = df['dry_food_brand_pre_tails'].astype(str).str.contains('diet', case=False, na=False).astype(int)

# B. 독립 변수 생성 

# 1. 질병 유무 ('has_health_issue')
df['has_health_issue'] = (df['pet_health_issue_list'].notna() & (df['pet_health_issue_list'] != 'None')).astype(int)

# 2. 칼로리 수준 ('is_low_kcal'): 0/1 이진 변수
mean_kcal = df['total_order_kcal'].mean()
df['is_low_kcal'] = (df['total_order_kcal'] < mean_kcal).astype(int)

# 3. 중성화 여부 ('neutered_y'): 0/1 이진 변수
df['neutered_y'] = df['neutered'].astype(str).str.upper().map({'TRUE': 1, 'FALSE': 0}).fillna(0).astype(int)

# 4. 웹 활동 변수 (스케일링)
continuous_cols = ['total_minutes_on_website', 'total_web_sessions']
scaler = StandardScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols].fillna(0)) 

# -------------------------- 로지스틱 회귀 모델 생성 및 실행 --------------------------

# [수정된 부분]
# 1. smf.logit 함수를 사용하여 포뮬러 방식을 적용합니다.
# 2. 변수 이름을 전처리된 이름(is_low_kcal, neutered_y)으로 정확히 일치시킵니다.
# 3. 0/1 이진 변수에는 C()를 사용하지 않습니다.

formula = "diet_y ~ has_health_issue + total_minutes_on_website + total_web_sessions + is_low_kcal + neutered"

# 로지스틱 회귀 모델 적합 (smf.logit 사용)
model = smf.logit(formula=formula, data=df)
result = model.fit(disp=False) 

# -------------------------- 결과 출력 및 해석 --------------------------

print("=======================================================================")
print("                   Diet 사료 급여 요인 분석 (로지스틱 회귀)")
print("=======================================================================")
print(result.summary())
print("\n")

print("---------------------------- Odds Ratio 해석 -----------------------------")

# Odds Ratio 계산
odds_ratios = np.exp(result.params)
odds_ratios = odds_ratios.drop('Intercept', errors='ignore') # smf.logit은 상수항을 'Intercept'로 표시
p_values = result.pvalues.drop('Intercept', errors='ignore')

odds_ratios_df = pd.DataFrame({
    'Odds Ratio (OR)': odds_ratios.round(3),
    'P-value': p_values.round(3)
})
odds_ratios_df.index.name = "독립 변수"

print("1. Odds Ratio (OR) 및 P-value")
print(odds_ratios_df)
print("\n")

print("2. 포뮬러: 'diet_y ~ has_health_issue + total_minutes_on_website + total_web_sessions + is_low_kcal + neutered_y'")
print("-----------------------------------------------------------------------")

                   Diet 사료 급여 요인 분석 (로지스틱 회귀)
                           Logit Regression Results                           
Dep. Variable:                 diet_y   No. Observations:                49042
Model:                          Logit   Df Residuals:                    49036
Method:                           MLE   Df Model:                            5
Date:                Tue, 28 Oct 2025   Pseudo R-squ.:                 0.05673
Time:                        16:48:37   Log-Likelihood:                -4377.1
converged:                       True   LL-Null:                       -4640.3
Covariance Type:            nonrobust   LLR p-value:                1.548e-111
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -5.2386      0.099    -52.956      0.000      -5.433      -5.045
neutered[T.True]             0.8750      0.

In [3]:
import pandas as pd
df = pd.read_csv('pet_food_customer_orders.csv')

mid_df=df[df['pet_food_tier']=='mid']
mid_df['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Bakers                               3598
Harringtons                          1632
Pedigree                             1510
Wagg                                  661
Beta                                  567
Skinner's Field & Trial               525
Asda Hero                             423
Tesco Premium                         300
Iams                                  286
Autarky                               174
Lifestage from Jollyes                167
Beneful                               161
Wagg Working Dog                      153
Perfect Fit                           147
Dr John                               144
Pets at Home Complete                 139
Chudleys                               98
Wellness                               84
Alpha                                  53
Country Kibble                         45
Chudleys Working Dog                   43
HiLife                                 40
Pero Premium                           37
CSJ      

In [4]:
pre_df=df[df['pet_food_tier']=='premium']
pre_df['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Harringtons                        2856
Wagg                               1593
Asda Hero                           835
Bakers                              708
Tesco Premium                       503
Pedigree                            487
Dr John                             326
Pets at Home Complete               283
Beneful                             283
Wagg Working Dog                    207
HiLife                              108
CSJ                                  88
Orlando                              84
Skinners                             83
Arkwrights                           68
Vitalin                              63
Alpha                                60
Wainwright's                         38
Sneyd's Wonderdog                    30
Arden Grange                         24
Supadog                              20
Pets at Home Moist Meaty Chunks      17
Feedwell                             15
Gelert Country Choice                13
Kirkland (Costc

In [9]:
pd.set_option('display.max_rows', None)
supre_df=df[df['pet_food_tier']=='superpremium']
supre_df['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Other / Autres                       3609
Royal Canin                          2560
James Wellbeloved                    2363
Wainwright's                         2134
AVA                                  1180
Iams                                 1109
Royal Canin Breed Specific            861
Lily's Kitchen                        721
Arden Grange                          704
Burns                                 665
Hills Science Plan                    557
Royal Canin Veterinary Diet           488
Eukanuba                              473
Purina One                            362
Barking Heads                         339
Canagan                               285
Hills Prescription Diet               230
Pets at Home Advanced Nutrition       208
PRO PLAN                              175
Acana                                 165
Purina Vet Diet                       154
Millies Wolfheart                     143
More                                  138
Eden     

In [30]:
df['signup_promo'].value_counts()

signup_promo
Null & Default      12059
Search Generic       8535
Digital Display      6364
Refer a Friend       5297
Incompletes          5248
Search Brand         4247
Events               2634
Affiliates           1906
Social Marketing      840
Inserts               832
Other                 762
Shopping Centres      184
Vet                   134
Name: count, dtype: int64

In [11]:
df['customer_id'].unique

<bound method Series.unique of 0        10574848487411271014
1        10574848487411271014
2        10574848487411271014
3        10574848487411271014
4        10574848487411271014
5         6342772975217424401
6         6342772975217424401
7         6342772975217424401
8         6342772975217424401
9         6342772975217424401
10        6342772975217424401
11        6342772975217424401
12        6342772975217424401
13        6342772975217424401
14        6342772975217424401
15        6342772975217424401
16        6018609623462647744
17        6018609623462647744
18        6018609623462647744
19        6018609623462647744
20        6018609623462647744
21        4432388076718555246
22        4432388076718555246
23        4432388076718555246
24        4432388076718555246
25        4432388076718555246
26         894531069061266030
27         894531069061266030
28         894531069061266030
29       17505316106326511065
30       17505316106326511065
31        6150681532737613937
32       

In [ ]:
df['customer_id'].value_counts()

customer_id
3224805444806377375     51
3944932459261526104     43
7471828410167071086     37
3847391468381334636     32
14435666773446139966    32
6365195295148646455     32
17491349881903470166    30
5370712501066328348     29
16084838255425143486    29
3745998493086947285     29
4101868794931697213     29
5113596527699749738     28
5968245914588929427     28
81635663551020700       28
496153615400193996      28
15643665125944591426    28
7512466054928015170     27
3555953331159086796     27
5082014266164986506     27
7234190264044082619     27
8189627308145546422     27
10459315942873124922    26
15476633212378420450    26
17436433834718744896    26
10922890556329147738    26
345036993219982508      25
3381825254933872998     25
2883551191945521469     25
15630939055109122921    25
12067826330552107837    25
8153239297783319382     25
7109621798169692433     25
1706858498887044672     25
17286811107269943802    25
17708523552721360892    25
17072113337211371275    25
1724584633235532

In [ ]:
id_3224805444806377375=df[df['customer_id']==3224805444806377375]
id_3224805444806377375['signup_promo']

In [ ]:
id_3944932459261526104=df[df['customer_id']==3944932459261526104]
id_3944932459261526104['signup_promo']

In [ ]:
id_7471828410167071086=df[df['customer_id']==7471828410167071086]
id_7471828410167071086['signup_promo']

In [37]:
id_4955304055506581926=df[df['customer_id']==4955304055506581926]
pd.set_option('display.max_columns', None)
id_4955304055506581926.sort_values(by='order_payment_date', ascending=False)

,customer_id,pet_id,pet_order_number,wet_food_order_number,orders_since_first_wet_trays_order,pet_has_active_subscription,pet_food_tier,pet_signup_datetime,pet_allergen_list,pet_fav_flavour_list,pet_health_issue_list,neutered,gender,pet_breed_size,signup_promo,ate_wet_food_pre_tails,dry_food_brand_pre_tails,pet_life_stage_at_order,order_payment_date,kibble_kcal,wet_kcal,total_order_kcal,wet_trays,wet_food_discount_percent,wet_tray_size,premium_treat_packs,dental_treat_packs,wet_food_textures_in_order,total_web_sessions,total_web_sessions_since_last_order,total_minutes_on_website,total_minutes_on_website_since_last_order,total_wet_food_updates,total_wet_food_updates_since_last_order,last_customer_support_ticket_date,customer_support_ticket_category
16725,4955304055506581926,5.587730e+18,10,8.0,8.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,20-3-12 0:00,38085.4215,3460.8,41546.2215,16,0.0,300g,0,0,gravy jelly,37,2,2375,780,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up
16723,4955304055506581926,5.587730e+18,9,7.0,7.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,20-2-14 0:00,38175.2150,3460.8,41636.0150,16,0.0,300g,0,0,gravy jelly,35,3,1595,0,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up
16717,4955304055506581926,5.587730e+18,8,6.0,6.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,20-1-16 0:00,38189.9805,3460.8,41650.7805,16,0.0,300g,0,0,gravy jelly,32,5,1595,1,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up
16720,4955304055506581926,5.587730e+18,3,1.0,1.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,half_maturity,19-8-20 0:00,44532.7525,3460.8,47993.5525,16,0.0,300g,0,0,gravy jelly,13,7,1472,143,0,0,2019-08-20 09:12:38+00:00,account
16721,4955304055506581926,5.587730e+18,2,NaN,NaN,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,half_maturity,19-8-1 0:00,44049.4000,0.0,44049.4000,0,NaN,NaN,0,0,NaN,6,4,1329,798,0,0,NaN,NaN
16724,4955304055506581926,5.587730e+18,1,NaN,NaN,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,half_maturity,19-7-15 0:00,25777.1010,0.0,25777.1010,0,NaN,NaN,0,0,NaN,2,1,531,499,0,0,NaN,NaN
16719,4955304055506581926,5.587730e+18,7,5.0,5.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,19-12-17 0:00,37106.9240,3460.8,40567.7240,16,0.0,300g,0,0,gravy jelly,27,3,1594,0,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up
16722,4955304055506581926,5.587730e+18,6,4.0,4.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,19-11-17 0:00,37091.2550,3460.8,40552.0550,16,0.0,300g,0,0,gravy jelly,24,5,1594,25,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up
16718,4955304055506581926,5.587730e+18,5,3.0,3.0,True,superpremium,00:01.6,NaN,NaN,NaN,True,male,large,Null & Default,True,Royal Canin,mature,19-10-17 0:00,37432.8220,3460.8,40893.6220,16,0.0,300g,0,0,gravy jelly,19,3,1569,0,0,0,2019-09-12 14:32:33+00:00,wet_food_top-up


In [40]:
df_weaning=df[df['pet_life_stage_at_order']=='weaning']
df_weaning['pet_food_tier'].value_counts()

pet_food_tier
superpremium    2172
mid              735
premium          480
Name: count, dtype: int64

In [48]:
df_weaning['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Other / Autres                       460
Pedigree                             307
Royal Canin                          307
Harringtons                          206
Bakers                               197
Iams                                 159
Wainwright's                         113
AVA                                  103
James Wellbeloved                     99
Skinner's Field & Trial               91
Wagg                                  87
Beta                                  59
Purina One                            58
Asda Hero                             55
Hills Science Plan                    50
Eukanuba                              47
Dr John                               45
Royal Canin Breed Specific            35
Lily's Kitchen                        35
(I don't currently feed dry food)     32
Tesco Premium                         31
Arden Grange                          25
PRO PLAN                              23
Chudleys                        

In [43]:
df_half_mature=df[df['pet_life_stage_at_order']=='half_maturity']
df_half_mature['pet_food_tier'].value_counts()

pet_food_tier
superpremium    5569
mid             1889
premium         1254
Name: count, dtype: int64

In [46]:
df_half_mature['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Royal Canin                          793
Other / Autres                       714
Harringtons                          684
Bakers                               584
Pedigree                             563
Wainwright's                         507
Iams                                 407
James Wellbeloved                    358
AVA                                  309
Wagg                                 259
Beta                                 179
Asda Hero                            168
Royal Canin Breed Specific           164
Hills Science Plan                   146
Eukanuba                             139
Lily's Kitchen                       131
Purina One                           120
Skinner's Field & Trial              117
Arden Grange                         112
Burns                                100
Pets at Home Complete                 82
Dr John                               78
Tesco Premium                         63
PRO PLAN                        

In [44]:
df_mature=df[df['pet_life_stage_at_order']=='mature']
df_mature['pet_food_tier'].value_counts()

pet_food_tier
superpremium    16646
mid              6612
premium          5875
Name: count, dtype: int64

In [47]:
df_mature['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Harringtons                          2884
Bakers                               2674
Other / Autres                       1963
James Wellbeloved                    1463
Wagg                                 1445
Wainwright's                         1283
Royal Canin                          1266
Pedigree                              911
Asda Hero                             767
AVA                                   707
Iams                                  668
Royal Canin Breed Specific            568
Lily's Kitchen                        493
Tesco Premium                         491
Arden Grange                          455
Burns                                 437
Beneful                               294
Royal Canin Veterinary Diet           289
Beta                                  282
Hills Science Plan                    277
Dr John                               269
Pets at Home Complete                 266
Wagg Working Dog                      264
Barking H

In [45]:
df_senior=df[df['pet_life_stage_at_order']=='senior']
df_senior['pet_food_tier'].value_counts()

pet_food_tier
superpremium    4144
mid             2282
premium         1384
Name: count, dtype: int64

In [49]:
df_senior['dry_food_brand_pre_tails'].value_counts()

dry_food_brand_pre_tails
Bakers                               851
Harringtons                          714
Other / Autres                       489
Wagg                                 463
James Wellbeloved                    443
Wainwright's                         279
Asda Hero                            268
Royal Canin                          221
Tesco Premium                        218
Pedigree                             216
Iams                                 169
Royal Canin Veterinary Diet          164
Arden Grange                         136
Beneful                              134
Burns                                113
Hills Science Plan                   108
Royal Canin Breed Specific            95
Hills Prescription Diet               91
Skinner's Field & Trial               86
AVA                                   80
Dr John                               78
Wagg Working Dog                      73
Lily's Kitchen                        72
Pets at Home Complete           